In [123]:
import base64

from requests import get
from fnmatch import fnmatch
from creds import github_url, github_token

In [124]:
def __parse_github_url(url):
    parts = url.strip("/").split("/")
    owner = parts[-2]
    repo = parts[-1]
    return owner, repo

In [125]:
def fetch_repo(url: str) -> list:
    """ Fetch the repository tree for the provided url """
    owner, repo = __parse_github_url(url)
    url = f"https://api.github.com/repos/{owner}/{repo}/git/trees/main?recursive=1"
    headers = {
        "Authorization": f"token {github_token}",
        "Accept": "application/vnd.github+json"
    }
    response = get(url, headers=headers)
    if response.status_code == 200:
        content = response.json()
        return content["tree"]
    else:
        raise ValueError(f"Error fetching repo contents: {response.status_code}")

In [126]:
def fetch_code(file_url: str) -> str:
    response = get(file_url)
    if response.status_code == 200:
        content = response.json()["content"]
        return base64.b64decode(content).decode('utf-8')

In [127]:
def get_pull_requests(url: str):
    owner, repo = __parse_github_url(url)
    url = f'https://api.github.com/repos/{owner}/{repo}/pulls'
    headers = {'Authorization': f'token {github_token}'}
    response = get(url, headers=headers)

    if response.status_code == 200:
        pr_data = response.json()
        return pr_data
    else:
        print(f"Error: Unable to fetch pull requests. Status code: {response.status_code}")
        return None

In [128]:
def get_pull_request_changes(pr_url: str):
    url = f'{pr_url}/files'
    headers = {'Authorization': f'token {github_token}'}
    response = get(url, headers=headers)

    if response.status_code == 200:
        return response.json()

In [129]:
def get_commit_history(url: str):
    owner, repo = __parse_github_url(url)
    url = f'https://api.github.com/repos/{owner}/{repo}/commits'
    headers = {'Authorization': f'token {github_token}'}
    response = get(url, headers=headers)

    if response.status_code == 200:
        commits_data = response.json()
        return commits_data
    else:
        print(f"Error: Unable to fetch commit history. Status code: {response.status_code}")
        return None

In [130]:
def get_commit_change(url: str, commit_sha: str):
    owner, repo = __parse_github_url(url)
    url = f'https://api.github.com/repos/{owner}/{repo}/commits/{commit_sha}'
    headers = {'Authorization': f'token {github_token}'}
    response = get(url, headers=headers)

    if response.status_code == 200:
        return response.json()


In [131]:
code_files = fetch_repo(github_url)

In [132]:
for file in code_files:
    if file["type"] == "blob" and fnmatch(file["path"], "*.py"):
        code = fetch_code(file['url'])
        print(code)

from flask import Flask, request
from google_api_connection import get_volumes, get_volume_info

app = Flask(__name__)

@app.get("/volumes")
def all_volumes():
    return get_volumes(request.args['q'])

@app.get("/volumes/<volume_id>")
def volume_info(volume_id: str) -> str:
    return get_volume_info(volume_id=volume_id)['volumeInfo']

""" A list of methods which integrate with Google APIs"""

import requests

BASE_URI = "https://www.googleapis.com/books/v1/volumes"


def get_volumes(query: str):
    """ Get list of items that match the requested query"""
    response = requests.get(f"{BASE_URI}?q={query}", timeout=1000)
    return response.json()


def get_volume_info(volume_id: str):
    """ Get specific information about the provided volume_id"""
    response = requests.get(f"{BASE_URI}/{volume_id}", timeout=1000)
    return response.json()

print(get_volume_info("t_ZYYXZq4RgC"))
    


In [133]:
commit_history = get_commit_history(github_url)
commit_change = get_commit_change(github_url, commit_history[2]['sha'])
print(commit_change['files'][1]['patch'])

@@ -1,4 +1,4 @@
-from flask import Flask, request, jsonify
+from flask import Flask, request
 from google_api_connection import get_volumes, get_volume_info
 
 app = Flask(__name__)
@@ -10,4 +10,3 @@ def all_volumes():
 @app.get("/volumes/<volume_id>")
 def volume_info(volume_id: str) -> str:
     return get_volume_info(volume_id=volume_id)['volumeInfo']
-


In [134]:
pr_history = get_pull_requests(github_url)
pr_changes = get_pull_request_changes(pr_history[0]['url'])
print(pr_changes[0]['patch'])

@@ -4,5 +4,3 @@
 
 * Get all volumes: ```/volumes```
 * Get Volume Info: ```/volumes/<volume_id>```
-    
-Adding something to test PR setup
